# Scraping du site de la SNEP : Top Albums hebdomadaire

Import de BeautifulSoup, requests, pandas, numpy et la classe Album

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from album import Album

Récupération des données du site

In [2]:
request_text = requests.get('https://snepmusique.com/les-tops/le-top-de-la-semaine/top-albums/')

Initialisation de BeautifulSoup

In [3]:
soup = BeautifulSoup(request_text.content, 'html.parser')

Création d'un DataFrame avec les colonnes relatives à un album

In [4]:
df = pd.DataFrame(columns=['rank','trend','title','artist','editor','last_week_rank','week_in','best_rank'])

df['rank'] = df['rank'].astype('int')
df['trend'] = df['trend'].astype('string')
df['title'] = df['title'].astype('string')
df['artist'] = df['artist'].astype('string')
df['editor'] = df['editor'].astype('string')
df['last_week_rank'] = df['last_week_rank'].astype('int')
df['week_in'] = df['week_in'].astype('int')
df['best_rank'] = df['best_rank'].astype('int')

df.dtypes

rank               int64
trend             string
title             string
artist            string
editor            string
last_week_rank     int64
week_in            int64
best_rank          int64
dtype: object

Bouclage sur la liste d'album du site, création des objets Album et du contenu du DataFrame

In [5]:
for element in soup.find_all('div', class_='item'):
    
    album = Album(
        element.find('div', class_='rang').get_text(),
        'Up' if(element.find('div', class_='rang_up icon-bigarrowup') != None) else ('Down' if(element.find('div', class_='rang_down icon-bigarrowdown') != None) else 'Neutral'),
        element.find('div', class_='titre').get_text(),
        element.find('div', class_='artiste').get_text(),
        element.find('div', class_='editeur').get_text(),
        -1 if element.find('div', class_='rang_precedent') == None else int(element.find('div', class_='rang_precedent').find('strong').get_text()[0:-2 if 'er' in element.find('div', class_='rang_precedent').find('strong').get_text() else -1]),
        0 if element.find('div', class_='week_in').find('strong').get_text() == 'Nouvelle entrée' else int(element.find('div', class_='week_in').find('strong').get_text()[0:-2 if 'er' in element.find('div', class_='week_in').find('strong').get_text() else -1]),
        int(element.find('div', class_='best_pos').find('strong').get_text()[0:-2 if 'er' in element.find('div', class_='best_pos').find('strong').get_text() else -1]),
    )
    
    df = df.append({
        'rank': album.rank,
        'trend': album.trend,
        'title': album.title,
        'artist': album.artist,
        'editor': album.editor,
        'last_week_rank': album.last_week_rank,
        'week_in': album.week_in,
        'best_rank': album.best_rank
    }, ignore_index=True)


Apperçu du dataFrame

In [6]:
df.head()

,rank,trend,title,artist,editor,last_week_rank,week_in,best_rank
0,1,Neutral,JEFE,NINHO,REC. 118 / MAL LUNE MUSIC,-1,0,1
1,2,Down,CIVILISATION,ORELSAN,3EME BUREAU,1,2,1
2,3,Down,30,ADELE,COLUMBIA,2,2,2
3,4,Neutral,NONANTE-CINQ,ANGÈLE,ROMANCE MUSIQUE / ANGELE VL RECORDS,-1,0,4
4,5,Down,=,ED SHEERAN,WEA / EAST WEST UK,4,5,1
